In [17]:
!pip install gym-jsbsim

In [18]:
import gym
import gym_jsbsim
from gym_jsbsim.catalogs.catalog import Catalog as c
import numpy as np
import math
import random
import torch
from torch import nn
import torch.nn.functional as F

In [19]:
class PolicyNetwork(nn.Module):
    def __init__(self,num_hidden_neurons):
        super(PolicyNetwork, self).__init__()
        self.layer1 = nn.Linear(9,num_hidden_neurons)
        self.layer2 = nn.Linear(num_hidden_neurons,4)

    def forward(self, x):
        x=torch.tanh(self.layer1(x))
        x=torch.tanh(self.layer2(x))
        return x

In [20]:
np.random.seed(0)
random.seed(0)
# init_models = []
# for i in range(5000):
#   torch.manual_seed(i)
#   model = PolicyNetwork(16)
#   for param in model.parameters():
#       param.requires_grad = False
#   init_models.append(model)
#init_center = np.concatenate((model.layer1.weight.data.numpy().flatten(),model.layer2.weight.data.numpy().flatten()))
torch.manual_seed(1453)
model = PolicyNetwork(16)
for param in model.parameters():
  param.requires_grad = False
init_center = np.concatenate((model.layer1.weight.data.numpy().flatten(),model.layer2.weight.data.numpy().flatten()))

In [21]:
def vec_to_weights(vec,num_hidden_neurons):
  w1 = torch.from_numpy(np.reshape(vec[0:num_hidden_neurons*9], (num_hidden_neurons, 9))).float()
  w2 = torch.from_numpy(np.reshape(vec[num_hidden_neurons*9:13*num_hidden_neurons], (4, num_hidden_neurons))).float()
  return w1,w2

In [22]:
class Obs_TupleToBoxWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        low = np.empty(shape=(0,))
        high = np.empty(shape=(0,))
        for i in env.observation_space:
          low = np.concatenate([low,i.low])
          high = np.concatenate([high,i.high])
        self.observation_space = gym.spaces.Box(low=low, high=high, dtype="float")
        
    
    def observation(self, obs):
        new_obs = np.empty(shape=(0,))
        for i in obs:
          new_obs = np.concatenate([new_obs,i])
        return new_obs

In [23]:
class Act_TupleToBoxWrapper(gym.ActionWrapper):
    def __init__(self, env):
        super().__init__(env)
        low = np.empty(shape=(0,))
        high = np.empty(shape=(0,))
        for i in env.action_space:
          low = np.concatenate([low,i.low])
          high = np.concatenate([high,i.high])
        self.action_space = gym.spaces.Box(low=low, high=high, dtype="float")
        
    
    def action(self, act):
#        print(act)
#        new_act = np.empty(shape=(0,))
#        for i in act:
#          new_act = np.concatenate([new_act,i])
        return act

In [24]:
# class Reward_Wrapper(gym.RewardWrapper):
#     def __init__(self, env):
#         super().__init__(env)
#         self.env = env
    
#     def reward(self, rew):
#         delta_heading = self.env.sim.get_property_value(c.delta_heading)
#         delta_heading_error_scale = 5 #degrees
#         heading_reward_scale = 0.3
#         if math.fabs(delta_heading) <= delta_heading_error_scale:
#             heading_r =  (heading_reward_scale/delta_heading_error_scale)*(delta_heading_error_scale- math.fabs(delta_heading))
#         else:
#             heading_r =  -1*heading_reward_scale

#         delta_altitude = self.env.sim.get_property_value(c.delta_altitude)
#         delta_altitude_error_scale = 50 #feet
#         altitude_reward_scale = 0.3
#         if math.fabs(delta_altitude) <= delta_altitude_error_scale:
#             alt_r =  (altitude_reward_scale/delta_altitude_error_scale)*(delta_altitude_error_scale- math.fabs(delta_altitude))
#         else:
#             alt_r =  -1*altitude_reward_scale

#         roll = self.env.sim.get_property_value(c.attitude_roll_rad) 
#         roll_error_scale = 0.35  # radians ~= 20 degrees
#         roll_reward_scale = 0.1
#         if math.fabs(roll) <= roll_error_scale:
#             roll_r =  (roll_reward_scale/roll_error_scale)*(roll_error_scale- math.fabs(roll))
#         else:
#             roll_r =  -1*roll_reward_scale

#         speed = self.env.sim.get_property_value(c.velocities_u_fps)
#         speed_error_scale = 16  # fps (~5%)
#         speed_reward_scale = 0.1
#         if math.fabs(speed-800) <= speed_error_scale:
#             speed_r =  (speed_reward_scale/speed_error_scale)*(speed_error_scale- math.fabs(speed-800))
#         else:
#             speed_r =  -1*speed_reward_scale        

#         # accel scale in "g"s
#         accel_error_scale_x = 0.1
#         accel_error_scale_y = 0.1
#         accel_error_scale_z = 0.5
#         accel_reward_scale = 0.2
#         accel_x = self.env.sim.get_property_value(c.accelerations_n_pilot_x_norm)
#         accel_y = self.env.sim.get_property_value(c.accelerations_n_pilot_y_norm)
#         accel_z = self.env.sim.get_property_value(c.accelerations_n_pilot_z_norm)
#         if math.fabs(accel_x) <= accel_error_scale_x:
#             accel_r_x =  (1/3)*(accel_reward_scale/accel_error_scale_x)*(accel_error_scale_x- math.fabs(accel_x))
#         else:
#             accel_r_x =  -(1/3)*accel_reward_scale

#         if math.fabs(accel_y) <= accel_error_scale_y:
#             accel_r_y =  (1/3)*(accel_reward_scale/accel_error_scale_y)*(accel_error_scale_y- math.fabs(accel_y))
#         else:
#             accel_r_y =  -(1/3)*accel_reward_scale

#         if math.fabs(accel_z+1) <= accel_error_scale_z:
#             accel_r_z =  (1/3)*(accel_reward_scale/accel_error_scale_z)*(accel_error_scale_z- math.fabs(accel_z+1))
#         else:
#             accel_r_z =  -(1/3)*accel_reward_scale        
#         accel_r = accel_r_x + accel_r_y + accel_r_z

#         reward = (heading_r + alt_r + accel_r + roll_r + speed_r)
#         return reward

In [25]:
env=Act_TupleToBoxWrapper(Obs_TupleToBoxWrapper(gym.make("GymJsbsim-HeadingControlTask-v0")))

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float64
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [26]:
def find_and_take_action(env,curr_state,nn):
  action = nn(torch.from_numpy(curr_state).float())
  action.numpy()
  action[-1]=(action[-1]+1)*0.45
  state, reward, done, _ = env.step(action)
  return state, reward, done

In [27]:
def run_episode(env,nn,find_and_take_action):
  episode_reward = 0
  state = env.reset()
  #print("Initial State =", state)
  done = False
  while not done:
    state, reward, done = find_and_take_action(env,state,nn)
    episode_reward += reward
    #print("action =", action, " ---> State =", state, " : Reward =", reward)
  #print("Simulation Time = "+"{:.3f}".format(env.get_sim_time())+" sec")
  #print("Episode Reward = "+"{:.3f}".format(episode_reward))
  if(env.get_sim_time()<150):
    objective = 0
  elif(env.get_sim_time()>150.0000000000115):
    objective = episode_reward
  else:
    objective = (1/abs(state[0]+1))*(1/abs(state[1]+1))
  return env.get_sim_time(),episode_reward,objective

In [28]:
# init_center_rewards = []
# init_center_times = []
# for i in range(5000):
#   init_center_reward, init_center_time = run_episode(env,init_models[i],find_and_take_action)
#   init_center_rewards.append(init_center_reward)
#   init_center_times.append(init_center_time)
# init_center_reward = max(init_center_rewards)
# id = init_center_rewards.index(init_center_reward)
# init_center_time = init_center_times[id]
# init_center = np.concatenate((init_models[id].layer1.weight.data.numpy().flatten(),init_models[id].layer2.weight.data.numpy().flatten()))

In [29]:
init_center_time, init_center_reward, init_center_objective = run_episode(env,model,find_and_take_action)

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float64
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [30]:
#muller-marsaglia method
def spherepicking(n):
    while True:           #to get rid off [0,0,0,0] case
        l = [random.gauss(0, 1) for i in range(n)]
        sumsq = sum([x * x for x in l])
        if sumsq > 0:
            break
    norm = 1.0 / math.sqrt(sumsq)
    pt = [x * norm for x in l]
    return np.array(pt)

In [31]:
def hill_climbing(env,spherepicking,lr,init_center,init_center_reward,init_center_time,init_center_objective,num_hidden_neurons,num_points,run_episode,find_and_take_action):

  models = []
  init_w1, init_w2 = vec_to_weights(init_center,num_hidden_neurons)
  for i in range(num_points):
    models.append(PolicyNetwork(num_hidden_neurons))
    for param in models[i].parameters():
      param.requires_grad = False

  center_is_best = False
  num_weights = num_hidden_neurons*9 + 4*num_hidden_neurons
  center = init_center
  reward = init_center_reward
  time = init_center_time
  objective = init_center_objective
  step = 0
  weights = []
  weights.append(center)
  print("Step = ",step)
  print("Simulation Time = ",time," sec")
  print("Reward = ",reward)
  print("Objective = ",objective)

  while not center_is_best:
    rewards = []
    sim_time = []
    objectives = []
    points = []
    for i in range(num_points):
      point = center + lr*spherepicking(num_weights)
      points.append(point)
      w1, w2 = vec_to_weights(point,num_hidden_neurons)
      models[i].layer1.weight.data = w1
      models[i].layer2.weight.data = w2
      t, r, o = run_episode(env,models[i],find_and_take_action)
      rewards.append(r)
      sim_time.append(t)
      objectives.append(o)
    max_objective = max(objectives)
    id = objectives.index(max_objective)
    if(max_objective<objective):
      center_is_best = True
      best_vec = center
    else:
      objective = max_objective
      time = sim_time[id]
      reward = rewards[id]
      step+=1
      print("Step = ",step)
      print("Simulation Time = ",time," sec")
      print("Reward = ",reward)
      print("Objective = ",objective)
      center = points[id]
      weights.append(center)
      new_w1, new_w2 = vec_to_weights(center,num_hidden_neurons)
      for i in range(num_points):
        models[i].layer1.weight.data = new_w1
        models[i].layer2.weight.data = new_w2
  return weights 

In [32]:
best_weights = hill_climbing(env,spherepicking,1,init_center,init_center_reward,init_center_time,init_center_objective,16,10000,run_episode,find_and_take_action)

Step =  0
Simulation Time =  150.0000000000115  sec
Reward =  648.3152019780708
Objective =  0.0008571788194772197


/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float64
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Step =  1
Simulation Time =  150.0000000000115  sec
Reward =  10.20408674663849
Objective =  0.0013528776859244257
Step =  2
Simulation Time =  150.0000000000115  sec
Reward =  116.6550296525484
Objective =  0.0029109212243582307
Step =  3
Simulation Time =  300.0000000000558  sec
Reward =  237.11174900614154
Objective =  237.11174900614154
Step =  4
Simulation Time =  619.8333333330983  sec
Reward =  415.7872914602629
Objective =  415.7872914602629


In [34]:
print(len(best_weights))

5


In [ ]:
best_weights

In [35]:
best_weights = hill_climbing(env,spherepicking,0.1,best_weights[4],415.7872914602629,619.8333333330983,415.7872914602629,16,10000,run_episode,find_and_take_action)

Step =  0
Simulation Time =  619.8333333330983  sec
Reward =  415.7872914602629
Objective =  415.7872914602629


/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float64
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Step =  1
Simulation Time =  468.6666666665691  sec
Reward =  1729.6067110989547
Objective =  1729.6067110989547
